<h1>Sythetic Data Generation</h1>
Create Dataset for Yolov8 <br>
The scene is comprised of clean, dirty plates and bottles which resembles the run time image entry.

In [ ]:
# jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import omni
from pxr import PhysicsSchemaTools, Gf, UsdGeom
import numpy as np
from omni.isaac.core.utils.stage import add_reference_to_stage
from omni.isaac.core.utils.prims import get_prim_at_path

<h2><font color='DimGray'>Load Stage</font></h2>

In [ ]:
root_dir = "/home/kijung914/myisaac/myfiles"

In [ ]:
# Opening a Stage
usd_context = omni.usd.get_context()
omni.kit.window.file.open_stage(f"{root_dir}/usd_files/stage.usd") #Load Stage

# WAIT for world to load!!

In [ ]:
stage = omni.usd.get_context().get_stage() # Obtain Stage

<h2><font color='DimGray'>How to spawn plates</font></h2>

In [ ]:
dirtyplate_file_path  = f"{root_dir}/usd_files/dirty_plate.usd"
cleanplate_file_path = f"{root_dir}/usd_files/clean_plate.usd"

dirtyplate_prim_path = "/dirty_plate"
cleanplate_prim_path = "/clean_plate"

bottle1_file_path  = f"{root_dir}/usd_files/bottle1.usd"
bottle2_file_path  = f"{root_dir}/usd_files/bottle2.usd"
bottle_prim_path = "/bottle"

In [ ]:
def spawn_dirty_plate():
    usd_path, prim_path = dirtyplate_file_path, dirtyplate_prim_path
    
    add_reference_to_stage(usd_path=usd_path, prim_path=prim_path) #Spawn
    
    xform = UsdGeom.Xformable(get_prim_at_path(prim_path)) # Apply necessary transforms
    scale_op = xform.AddScaleOp()
    rotate_op = xform.AddRotateXYZOp()
    translate_op = xform.AddTranslateOp()
    translate_op.Set(Gf.Vec3f(-5, 0, 1.8))

def spawn_clean_plate():
    usd_path, prim_path = cleanplate_file_path, cleanplate_prim_path
    
    add_reference_to_stage(usd_path=usd_path, prim_path=prim_path) #Spawn
    
    xform = UsdGeom.Xformable(get_prim_at_path(prim_path)) # Apply necessary transforms
    scale_op = xform.AddScaleOp()
    rotate_op = xform.AddRotateXYZOp()
    translate_op = xform.AddTranslateOp()
    translate_op.Set(Gf.Vec3f(-5, 0, 1.8))


def spawn_bottle(key):
    if key == 1:
        usd_path = bottle1_file_path
    else:
        usd_path = bottle2_file_path

    prim_path = bottle_prim_path
    
    add_reference_to_stage(usd_path=usd_path, prim_path=prim_path)#Spawn
    
    xform = UsdGeom.Xformable(get_prim_at_path(prim_path))# Apply necessary transforms to root xform
    scale_op = xform.AddScaleOp()
    translate_op = xform.AddTranslateOp()

    xform = UsdGeom.Xformable(get_prim_at_path(f"{prim_path}/Meshes")) # # Apply necessary transforms to mesh xform
    rotate_op = xform.AddRotateXYZOp()
    
    scale_op.Set(Gf.Vec3f(0.01,0.01,0.01))
    rotate_op.Set(Gf.Vec3f(0, 0, 0))

    translate_op.Set(Gf.Vec3f(-580, -4.5, 187))

In [ ]:
#spawn_dirty_plate()

In [ ]:
#spawn_clean_plate()

In [ ]:
#spawn_bottle(1)

<h2><font color='red'>Create Dataset with Replicator</font></h2>

In [ ]:
def simulate(p=1, dp=1, b=0, save='', nframes=10):
    import omni.replicator.core as rep
    import random
    rep.settings.set_render_pathtraced() # Set to path_traced

    with rep.new_layer(): # Create new layer
        camera = rep.create.camera(
            position=(-4.9872, 0.01507, 3.76027),
            look_at=(-4.9872, 0.01507, 0.0)
        ) # Create Camera
        render_product = rep.create.render_product(camera, (1024, 1024)) #Set render result as 1024x1024
        
        targets = [] # plates and bottles
        dirty_prims = [] # prims for randomizing how dirty plate looks
        bottle_orientation_prims = [] # prims for randomzing bottle orientation

        if dp > 0: # If the scene needs at least one dirty plate
            for i in range(dp):
                dirty_plate = rep.create.from_usd(dirtyplate_file_path) # Create dirty plate
                with dirty_plate:
                    rep.modify.pose(position=(-5, 0, 1.8), rotation=(0, 0, 0), scale=1.0) # Move
                    rep.modify.semantics([("class", "dirty_plate")]) # Create Semantic
                targets.append(dirty_plate) # Save to targets list

                # For randomzing dirty plate apperance
                dirty_prim = dirty_plate.get_output_prims()['prims'][0].GetPath().pathString
                
                dirty_light = rep.get.prims(
                    path_pattern=f"{dirty_prim}/Ref/myroot/dirty_light_out/dirty*",
                    prim_types=["Xform"]
                ) # For "dirty sauce" effect
                dirty_blob = rep.get.prims(
                    path_pattern=f"{dirty_prim}/Ref/myroot/items/bl*",
                    prim_types=["Xform"]
                ) # For "dirty chunck" effect
                
                dirty_myplate = rep.get.prims(
                    path_pattern=f"{dirty_prim}/Ref/myroot",
                    prim_types=["Xform"]
                ) # For flipping <- Was not used

                dirty_prims.append([dirty_light, dirty_blob, dirty_myplate]) # Save

        if p > 0: # If the scene needs at least one clean plate
            for _ in range(p):
                clean_plate = rep.create.from_usd(cleanplate_file_path) # Create clean plate
                with clean_plate:
                    rep.modify.pose(position=(-5, 0, 1.8), rotation=(0, 0, 0), scale=1.0) # Move
                    rep.modify.semantics([("class", "clean_plate")]) # Create semantic

                targets.append(clean_plate) # Save to targets list
                #clean plate does not need appearance randomization

        if b > 0: # If the scene needs at least one bottle
            choices = [1,2]
            for _ in range(b):
                key = random.choice(choices) # Choose between two bottle types
                if key == 1:
                    bottle = rep.create.from_usd(bottle1_file_path) # Spawn type 1 bottle
                else:
                    bottle = rep.create.from_usd(bottle2_file_path) # Spawn type 2 bottle

                with bottle:
                    rep.modify.pose(position=(-5, -0, 1.8), rotation=(0,0,0), scale=0.01) # Move
                targets.append(bottle) # Save to targets list

                # For bottle orientation randomization
                bottle_path = bottle.get_output_prims()['prims'][0].GetPath().pathString
                bottle_mesh = rep.get.prims(
                    path_pattern=f"{bottle_path}/Ref/Mesh*",
                    prim_types=["Xform"]
                )

                bottle_orientation_prims.append(bottle_mesh) # Save


        # Replicator Frame Generation!!
        with rep.trigger.on_frame(num_frames=nframes): # create n amount of frames
            if dp > 0:
                for i in range(dp):
                    # Flipping causes wrong teleport, so was not used
                    #flip_list = [(180,0,0),(180,0,0)]
                    #with dirty_myplate:
                    #   rep.modify.pose(rotation=random.choice(flip_list))
                    
                    # Change light source -> change 'dirty sauce' effect
                    dirty_light = dirty_prims[i][0]
                    with dirty_light:
                        rep.modify.pose(rotation=rep.distribution.uniform((-20,-20,-90),(20,20,90)))

                    # Change 'dirty chunk'
                    dirty_blob = dirty_prims[i][1]
                    with dirty_blob:
                        rep.modify.pose(position=rep.distribution.uniform((-0.08,-0.08,0),(0.08,0.08,0)))

            if b > 0:
                for i in range(b):
                    # Change bottle orientation
                    bottle_mesh = bottle_orientation_prims[i]
                    with bottle_mesh:
                        rep.modify.pose(rotation=rep.distribution.uniform((0,0,-180),(0,0,180)))

            # With all the plates and bottles, change position
            with rep.create.group(targets):
                rep.modify.pose(
                    position=rep.distribution.uniform((-5.65, -0.2, 1.8), (-4.34, 0.18, 1.95)))

        # Save rendered image
        if save=='coco': # if cocowriter
            coco = rep.WriterRegistry.get("CocoWriter")
            coco.initialize(
                output_dir="_output",
                category_to_labels={
                    "dirty_plate": ["dirty_plate"], #"plate": ["dirty_plate", "clean__plate"]로 묶을 수 있음
                    "clean_plate": ["clean_plate"]
                },
                json_file_name="instances_train.json",
                include_images=True
            )
            coco.attach([render_product])
        elif save=='basic': # if basicwriter
            writer = rep.WriterRegistry.get("BasicWriter")
            writer.initialize( output_dir="_output", rgb=True,   bounding_box_2d_tight=True)
            writer.attach([render_product])

        # Begin
        rep.orchestrator.run()

In [ ]:
simulate(p=1, dp=1, b=7, nframes=1300, save='basic') # clean 1, dirty 1, bottle 7, 1300 shots